> Given a list of numbers and a number `k`, return whether any two numbers from the list add up to k.
For example, given `[10, 15, 3, 7]` and `k` of 17, return `true` since 10 + 7 is 17.

# Basic
The following is a simple, not very efficient solution.
Time complexity : O(N²)

In [1]:
num_list = [10, 15, 3, 7]
target = 17

def sum_to_target(num_list: list, target: int) -> bool:
    for i in num_list:
        for j in num_list:
            if ((i + j) == target):
                return True
    return False
        
print(sum_to_target(num_list, target))

True


In [2]:
# Testing the above function

import random

allowed_range = (-10, 10)
print("List | Target | Result")
for i in range(10):
    new_list = [random.randint(*allowed_range) for x in range(5)]
    target = random.randint(*allowed_range)
    print(new_list, target, sum_to_target(new_list, target), sep = " | ")

List | Target | Result
[8, 2, -7, 8, -6] | 0 | False
[-3, 4, -7, -9, -9] | 8 | True
[-2, 5, -3, -1, 10] | -3 | True
[1, 0, -8, 10, 3] | -6 | False
[10, 3, -1, 3, 7] | -4 | False
[8, -2, 4, 0, -3] | -5 | True
[-2, -1, -1, -4, 8] | -8 | True
[-10, 9, -5, 5, 6] | 0 | True
[2, 0, -6, -7, 2] | 1 | False
[-8, 8, -5, -2, -9] | 8 | False


# Reducing possibilities 
One way to improve the program's efficiency would be to filter the possibilities.

In [3]:
# First, let's see what it looks like in a table.

import numpy as np

num_list = [10, 15, 3, 7]
num_list.sort()
target = 17

p = np.zeros((len(num_list) + 1, len(num_list) + 1)).astype(int)

# Sets first row and first column
p[1:, 0] = p[0, 1:] = num_list

for i in range(len(num_list) + 1)[1:]: # For each row
    for j in range(len(num_list) + 1)[1:]: # For each column
        # We sum up the values
        p[i, j] = p[i, 0] + p[0, j]

# What we have now is a table with each cell representing the sum of 
# the corresponding row and column.
print(p)

[[ 0  3  7 10 15]
 [ 3  6 10 13 18]
 [ 7 10 14 17 22]
 [10 13 17 20 25]
 [15 18 22 25 30]]


Alright. So, as we can see, the up-left and bottom-right corners are kind of useless : they contain way too large or way too small values ; we do not need to compute all of them to get our answer.

What we can do instead is to start the loop in the top-right corner (bottom-left would work as well) and go left if the value is superior to the target, down if the value is inferior.


Let's put that into code.

In [4]:
import numpy as np

num_list = [9, 12, 15, 11, 3]
target = 17

def sum_to_target(num_list: list, target: int):
    num_list.sort()
    p = np.zeros((len(num_list) + 1, len(num_list) + 1)).astype(int)
    # Sets first row and first column
    p[1:, 0] = p[0, 1:] = num_list
    
    i = 1 # We start from row 1
    for j in range(len(num_list), -1, -1): # For each column, starting from the end
        while True:
            value = p[i, 0] + p[0, j]
            p[i, j] = value
            if value < target:
                if i < len(num_list):
                    i += 1
                else:
                    return p
            else:
                break
        if value >= target:
            continue
    # This return is necessary in case we have a value larger than the target in the list
    return p 

print(sum_to_target(num_list, target))

[[ 0  3  9 11 12 15]
 [ 3  0  0  0 15 18]
 [ 9 12 18 20 21  0]
 [11 14  0  0  0  0]
 [12 15  0  0  0  0]
 [15 18  0  0  0  0]]


Great !

From what we can see, the script is processing the edge between the closest too large and too small values. We can also see is that the array is symmetrical, meaning we do not need to process it all ; only running through half will do the trick.

Cool, let's modify the function so it answers the question correctly.

In [5]:
import numpy as np

num_list = [10, 12, 15, 11, 3, 7]
target = 17

def sum_to_target_filt(num_list: list, target: int) -> bool:
    num_list.sort()
    p = np.zeros((len(num_list) + 1, len(num_list) + 1)).astype(int)

    # Sets first row and first column
    p[1:, 0] = p[0, 1:] = num_list
    
    i = 1 # We start from row 1
    # For each column, starting from the end
    for j in range(len(num_list), -1, -1):
        while True:
            if i + (len(num_list) - j) > len(num_list):
                return False
            value = p[i, 0] + p[0, j]
            p[i, j] = value
            if value < target:
                if i < len(num_list):
                    i += 1
                else:
                    return False
            else:
                break
        if value > target:
            continue
        if value == target:
            return True
    return False


print(sum_to_target_filt(num_list, target))

True


In [6]:
# Testing the above function

import random

allowed_range = (-10, 10)
print("List | Target | Result")
for i in range(10):
    new_list = [random.randint(*allowed_range) for x in range(5)]
    target = random.randint(*allowed_range)
    print(new_list, target, sum_to_target_filt(new_list, target), sep = " | ")

List | Target | Result
[-5, -3, -2, 0, 4] | 9 | False
[-9, 0, 8, 9, 10] | -8 | False
[-10, -5, -2, 0, 7] | 8 | False
[-10, -10, -8, -6, -6] | -1 | False
[-6, -3, 0, 8, 8] | -6 | True
[-8, 0, 2, 8, 10] | 2 | True
[2, 4, 4, 7, 9] | 2 | False
[-9, -9, 1, 2, 5] | 4 | True
[-2, -2, 2, 5, 7] | -6 | False
[-5, 2, 3, 8, 10] | -6 | False


# Conclusion
I'm pretty happy with the result, and I can't think of a way to improve the system even more.

The final time complexity is O(N * log(N) / 2).